In [1]:
import streamlit as st
import pandas as pd
import requests
from datetime import datetime
from io import StringIO

from modules.get_elo_data import get_elo_data
from modules.get_fixtures import get_fixtures

# Page config
st.set_page_config(page_title="Czech Football Clubs ELO", layout="wide")


2025-02-18 10:17:47.490 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
club_mapping = {
    62: {"tm_name": "sk-slavia-prag", "elo_api_name": "SlaviaPraha", "club_name": "SK Slavia Praha"},
    5546: {"tm_name": "fk-mlada-boleslav", "elo_api_name": "MladaBoleslav", "club_name": "FK Mladá Boleslav"},
    715: {"tm_name": "fc-bohemians-prag-1905", "elo_api_name": "BohemiansPraha", "club_name": "Bohemians 1905"},
    1496: {"tm_name": "fk-pardubice", "elo_api_name": "Pardubice", "club_name": "FK Pardubice"},
    377: {"tm_name": "fc-banik-ostrau", "elo_api_name": "BanikOstrava", "club_name": "FC Baník Ostrava"},
    197: {"tm_name": "ac-sparta-prag", "elo_api_name": "SpartaPraha", "club_name": "AC Sparta Praha"},
    1322: {"tm_name": "fk-jablonec", "elo_api_name": "Jablonec", "club_name": "FK Jablonec"},
    1897: {"tm_name": "fc-hradec-kralove", "elo_api_name": "HradecKralove", "club_name": "FC Hradec Králové"},
    13726: {"tm_name": "mfk-karvina", "elo_api_name": "Karvina", "club_name": "MFK Karviná"},
    1321: {"tm_name": "sk-dynamo-ceske-budejovice", "elo_api_name": "CeskeBudejovice", "club_name": "SK Dynamo České Budějovice"},
    814: {"tm_name": "fk-teplice", "elo_api_name": "Teplice", "club_name": "FK Teplice"},
    697: {"tm_name": "fc-slovan-liberec", "elo_api_name": "SlovanLiberec", "club_name": "FC Slovan Liberec"},
    941: {"tm_name": "fc-viktoria-pilsen", "elo_api_name": "ViktoriaPlzen", "club_name": "FC Viktoria Plzeň"},
    2311: {"tm_name": "sk-sigma-olmutz", "elo_api_name": "SigmaOlomouc", "club_name": "SK Sigma Olomouc"},
    450: {"tm_name": "fk-dukla-prag", "elo_api_name": "Dukla", "club_name": "FK Dukla Praha"},
    5544: {"tm_name": "1-fc-slovacko", "elo_api_name": "Slovacko", "club_name": "1. FC Slovácko"}
}

czech_clubs = pd.DataFrame.from_dict(club_mapping, orient="index").reset_index().rename(columns={"index": "club_id"})


In [3]:
fixtures = get_fixtures(2024, czech_clubs, club_mapping)

In [158]:
# Create a list to store club data
df_club_elo = []

# Fetch data for each club
for club in czech_clubs["elo_api_name"]:
    elo, date, club_elo_df = get_elo_data(club)
    if elo is not None:
        df_club_elo.append({
            'elo_api_name': club,
            'elo_rating': round(elo),
            'updated_at': date
        })

df_club_elo = pd.DataFrame(df_club_elo)


In [ ]:
elo_to_club = {entry["elo_api_name"]: entry["club_name"] for entry in club_mapping.values()}
club_elo_df["Club"] = club_elo_df["Club"].map(elo_to_club)
df_club_elo["club_name"] = df_club_elo["elo_api_name"].map(elo_to_club)

fixtures_clubs = {}
club_average_opponent_elo = pd.DataFrame(columns=["club_name", "average_home_opponent_elo", "average_away_opponent_elo", "average_opponent_elo"])

for club in czech_clubs["club_name"]:
    fixtures_clubs[club] = fixtures[(fixtures["home_team"] == club) | (fixtures["away_team"] == club)]
    fixtures_clubs[club] = fixtures_clubs[club].sort_values(by="event_timestamp")
    fixtures_clubs[club]["home_away"] = fixtures_clubs[club].apply(lambda x: "home" if x["home_team"] == club else "away", axis=1)
    fixtures_clubs[club]["opponent_elo"] = fixtures_clubs[club].apply(lambda x: df_club_elo[df_club_elo["club_name"] == x["away_team"]]["elo_rating"].values[0] if x["home_away"] == "home" else df_club_elo[df_club_elo["club_name"] == x["home_team"]]["elo_rating"].values[0], axis=1)
    
    future_fixtures = fixtures_clubs[club][fixtures_clubs[club]["is_planned_tf"]]

    average_home_opponent_elo = future_fixtures[future_fixtures['home_away'] == 'home']['opponent_elo'].mean()
    average_away_opponent_elo = future_fixtures[future_fixtures['home_away'] == 'away']['opponent_elo'].mean()
    average_opponent_elo = future_fixtures['opponent_elo'].mean()

    club_average_opponent_elo.loc[len(club_average_opponent_elo)] = [
        club,
        average_home_opponent_elo,
        average_away_opponent_elo,
        average_opponent_elo
    ]

club_average_opponent_elo.sort_values(by="average_opponent_elo", ascending=False, inplace=True)

In [ ]:

# App title
st.title("Czech Football Clubs ELO Ratings")

# Display stats
st.subheader("Current ELO Ratings")
st.dataframe(
    clubs_df,
    hide_index=True,
    column_config={
        'ELO Rating': st.column_config.NumberColumn(format="%d"),
        'Last Updated': st.column_config.DateColumn('Last Updated')
    }
)

# Calculate and display some statistics
st.subheader("League Statistics")
col1, col2, col3 = st.columns(3)

with col1:
    st.metric("Highest ELO", f"{clubs_df['ELO Rating'].max()}")
    st.caption(f"({clubs_df.iloc[0]['Club']})")
    
with col2:
    st.metric("Average ELO", f"{round(clubs_df['ELO Rating'].mean())}")
    
with col3:
    st.metric("Lowest ELO", f"{clubs_df['ELO Rating'].min()}")
    st.caption(f"({clubs_df.iloc[-1]['Club']})")

# Create a bar chart
st.subheader("ELO Ratings Comparison")
chart_data = clubs_df.set_index('Club')
st.bar_chart(chart_data['ELO Rating'])